In [66]:
#load the libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import altair as alt


In [67]:
root_path = 'dataset/'

customers_df = pd.read_csv(root_path + 'olist_customers_dataset.csv')
items_df = pd.read_csv(root_path + 'olist_order_items_dataset.csv')
payments_df = pd.read_csv(root_path + 'olist_order_payments_dataset.csv')
orders_df = pd.read_csv(root_path + 'olist_orders_dataset.csv')
products_df = pd.read_csv(root_path + 'olist_products_dataset.csv')
sellers_df = pd.read_csv(root_path + 'olist_sellers_dataset.csv')
categories_df = pd.read_csv(root_path + 'product_category_name_translation.csv')

Merging the 6 datasets to make a single dataframe

In [68]:
dataframes = {'customers': customers_df,
              'items': items_df, 
              'payments': payments_df, 
              'orders': orders_df, 
              'products': products_df, 
              'sellers': sellers_df, 
              'categories': categories_df}
for i, j in dataframes.items():
    print(f'{i:12s} dataframe: {str(len(j)):7s} rows')

customers    dataframe: 99441   rows
items        dataframe: 112650  rows
payments     dataframe: 103886  rows
orders       dataframe: 99441   rows
products     dataframe: 32951   rows
sellers      dataframe: 3095    rows
categories   dataframe: 71      rows


In [69]:
#product_df contains 73 unique categories in portuquese language, while categories_df english translation to the portuguese categories has 71
#the missing categories translation will be maintained in original portuguese language.

#products_df = pd.merge(products_df, categories_df, on='product_category_name', how='left')
# Delete 'product_category_name' column
#del products_df['product_category_name']
# Delete  the categories_df dataframe
#del categories_df
# Rename the column
#products_df.rename(columns={'product_category_name_english': 'product_category'}, inplace=True)

In [70]:
# Merge and clean up the products DataFrame
products_df = pd.merge(products_df, categories_df, on='product_category_name', how='left')
products_df.drop('product_category_name', axis=1, inplace=True)
products_df.rename(columns={'product_category_name_english': 'product_category'}, inplace=True)

In [71]:
# Update the 'products' key in the dataframes dictionary with the merged DataFrame
dataframes['products'] = products_df

In [72]:
for name, df in dataframes.items():
    print(f'\n{name} dataframe:')
    print(df.head(5))
    print('-' * 40)



customers dataframe:
                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3  b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4  4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   

   customer_zip_code_prefix          customer_city customer_state  
0                     14409                 franca             SP  
1                      9790  sao bernardo do campo             SP  
2                      1151              sao paulo             SP  
3                      8775        mogi das cruzes             SP  
4                     13056               campinas             SP  
----------------------------------------

items dataframe:
                           order_id  order_item_id  \
0  00010242fe8c5a6

In [73]:
# Merge DataFrames 
# (assuming you've already merged products_df and categories_df within the 'products' key)
orders_customers_df = pd.merge(dataframes['orders'], dataframes['customers'], on='customer_id')
orders_customers_items_df = pd.merge(orders_customers_df, dataframes['items'], on='order_id')
main_df = pd.merge(orders_customers_items_df, dataframes['payments'], on='order_id')
main_df = pd.merge(main_df, dataframes['products'], on='product_id')
main_df = pd.merge(main_df, dataframes['sellers'], on='seller_id')

In [74]:
# 2. Display the Data
print("Shape of the merged DataFrame:", main_df.shape)
print("\nMerged DataFrame - First 5 rows:")
print(main_df.head().to_markdown(index=False, numalign="left", stralign="left"))
print("\nMerged DataFrame - Column Info:")
print(main_df.info())

Shape of the merged DataFrame: (117601, 33)

Merged DataFrame - First 5 rows:
| order_id                         | customer_id                      | order_status   | order_purchase_timestamp   | order_approved_at   | order_delivered_carrier_date   | order_delivered_customer_date   | order_estimated_delivery_date   | customer_unique_id               | customer_zip_code_prefix   | customer_city   | customer_state   | order_item_id   | product_id                       | seller_id                        | shipping_limit_date   | price   | freight_value   | payment_sequential   | payment_type   | payment_installments   | payment_value   | product_name_lenght   | product_description_lenght   | product_photos_qty   | product_weight_g   | product_length_cm   | product_height_cm   | product_width_cm   | product_category   | seller_zip_code_prefix   | seller_city    | seller_state   |
|:---------------------------------|:---------------------------------|:---------------|:----------------------

In [75]:
# 3. Basic EDA
print("\nDescriptive statistics of numerical columns:")
print(main_df.describe().to_markdown(numalign="left", stralign="left"))

print("\nNumber of unique values and most frequent values for categorical columns:")
for col in main_df.select_dtypes(include='object'):
    print(f"\nColumn: {col}")
    print(main_df[col].value_counts().head(5).to_markdown(numalign="left", stralign="left"))


Descriptive statistics of numerical columns:
|       | customer_zip_code_prefix   | order_item_id   | price   | freight_value   | payment_sequential   | payment_installments   | payment_value   | product_name_lenght   | product_description_lenght   | product_photos_qty   | product_weight_g   | product_length_cm   | product_height_cm   | product_width_cm   | seller_zip_code_prefix   |
|:------|:---------------------------|:----------------|:--------|:----------------|:---------------------|:-----------------------|:----------------|:----------------------|:-----------------------------|:---------------------|:-------------------|:--------------------|:--------------------|:-------------------|:-------------------------|
| count | 117601                     | 117601          | 117601  | 117601          | 117601               | 117601                 | 117601          | 115903                | 115903                       | 115903               | 117581             | 117581              

In [77]:
# 1. Count of orders per category
product_category_counts = main_df['product_category'].value_counts().sort_values(ascending=False)

In [78]:
# 2. Distribution of order statuses
order_status_counts = main_df['order_status'].value_counts()

In [79]:
# 3. Count of orders per customer
orders_per_customer = main_df.groupby('customer_unique_id')['order_id'].nunique()

# 4. Count of each payment type
payment_type_counts = main_df['payment_type'].value_counts().sort_values(ascending=False)

# 5. Count of each installment option
installments_counts = main_df['payment_installments'].value_counts().sort_values(ascending=False)

# 6. Calculate order value
main_df['order_value'] = main_df['price'] + main_df['freight_value']

# 7. Average order value per category
avg_order_value_per_category = main_df.groupby('product_category')['order_value'].mean().sort_values(ascending=False)

# 8. Overall average order value
overall_avg_order_value = main_df['order_value'].mean()

# Display the outputs
print("\nTop 10 Product Categories by Order Count:")
print(product_category_counts.head(10).to_markdown(numalign="left", stralign="left"))

print("\nOrder Status Counts:")
print(order_status_counts.to_markdown(numalign="left", stralign="left"))

print("\nOrders per Customer Statistics:")
print(orders_per_customer.describe().to_markdown(numalign="left", stralign="left"))

print("\nPayment Type Counts:")
print(payment_type_counts.to_markdown(numalign="left", stralign="left"))

print("\nInstallment Counts:")
print(installments_counts.to_markdown(numalign="left", stralign="left"))

print("\nAverage Order Value per Category:")
print(avg_order_value_per_category.head(10).to_markdown(numalign="left", stralign="left"))

print("\nOverall Average Order Value:", overall_avg_order_value)


Top 10 Product Categories by Order Count:
| product_category      | count   |
|:----------------------|:--------|
| bed_bath_table        | 11823   |
| health_beauty         | 9972    |
| sports_leisure        | 8945    |
| furniture_decor       | 8744    |
| computers_accessories | 8082    |
| housewares            | 7355    |
| watches_gifts         | 6201    |
| telephony             | 4721    |
| garden_tools          | 4574    |
| auto                  | 4379    |

Order Status Counts:
| order_status   | count   |
|:---------------|:--------|
| delivered      | 115035  |
| shipped        | 1244    |
| canceled       | 566     |
| processing     | 375     |
| invoiced       | 371     |
| unavailable    | 7       |
| approved       | 3       |

Orders per Customer Statistics:
|       | order_id   |
|:------|:-----------|
| count | 95419      |
| mean  | 1.03402    |
| std   | 0.211235   |
| min   | 1          |
| 25%   | 1          |
| 50%   | 1          |
| 75%   | 1          |
| 